In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dsTrain = pd.read_csv("/kaggle/input/titanic/train.csv")
dsTest = pd.read_csv("/kaggle/input/titanic/test.csv")
dsTest2 = pd.read_csv("/kaggle/input/titanic/test.csv")
dsTest.isnull().sum()

In [ ]:
dsTrain.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer

colToDrop = ["Cabin", "Name", "Ticket", "PassengerId"]
dsTrain.drop(columns = colToDrop, inplace = True)
dsTest.drop(columns = colToDrop, inplace = True)

si = SimpleImputer(strategy = "mean")
dsTrain[["Age"]] = si.fit_transform(dsTrain[["Age"]])
dsTest[["Age"]] = si.fit_transform(dsTest[["Age"]])
dsTest[["Fare"]] = si.fit_transform(dsTest[["Fare"]])

dsTrain = dsTrain.dropna()
dsTest = dsTest.dropna()
dsTest

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

dsTrainY = dsTrain.iloc[:, 0].values
dsTrainX = dsTrain.iloc[:, 1:].values
dsTest.isnull().sum()

In [ ]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers = [("encoder", OneHotEncoder(), [1,6])], remainder = "passthrough")
dsTrainX = ct.fit_transform(dsTrainX)
dsTest = ct.fit_transform(dsTest)
pd.DataFrame(dsTest).isnull().sum()

In [ ]:
pd.DataFrame(dsTrainY)

In [ ]:
from sklearn.preprocessing import StandardScaler

scTrain = StandardScaler()
scTest = StandardScaler()

dsTrainX[:, 5] = scTrain.fit_transform(dsTrainX[:, 5].reshape(-1, 1)).flatten()
dsTrainX[:, 9] = scTrain.fit_transform(dsTrainX[:, 9].reshape(-1, 1)).flatten()
dsTrainX[:, 6] = scTrain.fit_transform(dsTrainX[:, 6].reshape(-1, 1)).flatten()
dsTest[:, 5] = scTest.fit_transform(dsTest[:, 5].reshape(-1, 1)).flatten()
dsTest[:, 9] = scTest.fit_transform(dsTest[:, 9].reshape(-1, 1)).flatten()
dsTest[:, 6] = scTest.fit_transform(dsTest[:, 6].reshape(-1, 1)).flatten()

print(pd.DataFrame(dsTest))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=42)
rfr.fit(dsTrainX, dsTrainY)

In [ ]:
yPred = rfr.predict(dsTest)
yPred = [0 if val<0.5 else 1 for val in yPred]
pd.DataFrame(yPred)

In [ ]:
dsMerged = pd.concat([dsTest2["PassengerId"], pd.DataFrame(yPred)], ignore_index=True, sort=True, axis = 1)
dsMerged.columns = ["PassengerId", "Survived"]
dsMerged.to_csv("submission.csv", index = False)
print("Hello")